# 3动态规划算法

定义本章所用环境:

In [7]:
import gym
import numpy as np

In [8]:
env = gym.make('FrozenLake-v1')
env.render()
a = env.observation_space
print(env.P[1][1])


SFFF
FHFH
FFFH
HFFG
[(0.3333333333333333, 0, 0.0, False), (0.3333333333333333, 5, 0.0, True), (0.3333333333333333, 2, 0.0, False)]


## 3.1Value迭代算法

策略迭代算法主要由两个模块组成： 
- 通过迭代计算optimal value function.  
- 根据得到的value function 提取策略（policy）

#### 3.1.1 computing the value function

In [16]:
def value_iteration(env, num_iter):
    threshold = 1e-10
    gamma = 1
    #初始化value function
    value_table = np.zeros(env.observation_space.n) #该问题中value function是一个table
    
    for i in range(num_iter):
        old_value_table = np.copy(value_table) #为了保留上一步的value_table 我们要新建一个table来进行迭代更新
        for s in range(env.observation_space.n):
            Q_values = [] #记录s状态下的所有action的Q（s，a）
            for a in range(env.action_space.n):
                q_values = 0
                for prob, s_, r, done in env.P[s][a]:
                    q_values += prob * (r + gamma * value_table[s_])
                Q_values.append(q_values)
            value_table[s] = max(Q_values)  #Q与V的关系
        if sum(np.fabs(value_table - old_value_table)) <= threshold:
            break
    return value_table

#### 3.1.2 extract the optimal policy

In [9]:
def extract_policy(value_table):
    gamma = 1.0
    policy = np.zeros(env.observation_space.n)
    
    for s in range(env.observation_space.n):
        Q_values = [] #记录s状态下的所有action的Q（s，a）
        for a in range(env.action_space.n):
            q_values = 0
            for prob, s_, r, done in env.P[s][a]:
                q_values += prob * (r + gamma * value_table[s_])
            Q_values.append(q_values)
        policy[s] = np.argmax(np.array(Q_values))
    return policy        

In [24]:
V = value_iteration(env, 1000)
policy = extract_policy(V)
print('value function is:', V, sep='\n')
print('policy is:', policy, sep='\n')

value function is:
[0.82352941 0.82352941 0.82352941 0.82352941 0.82352941 0.
 0.52941176 0.         0.82352941 0.82352941 0.76470588 0.
 0.         0.88235294 0.94117647 0.        ]
policy is:
[0. 3. 3. 3. 0. 0. 0. 0. 3. 1. 0. 0. 0. 2. 1. 0.]


## 3.2Policy迭代算法

policy迭代算法主要由以下四步组成:  
- 初始化policy  
- 使用所给的policy计算value function  
- 使用所得的value function计算新的policy  
- 判断现有policy是否为optimal policy

In [23]:
'''计算value function'''
def compute_value_fn(policy):
    num_iter = 1000
    threshold = 1e-20
    gamma = 1
    value_table = np.zeros(env.observation_space.n)
    
    for i in range(num_iter):
        old_value_table = np.copy(value_table)
        for s in range(env.observation_space.n):
            a = policy[s]
            value_table[s] = sum([prob * (r + gamma * value_table[s_]) for prob, s_, r, _ in env.P[s][a]])
        if (np.sum(np.fabs(old_value_table - value_table)) <= threshold):
            break
    return value_table

In [24]:
'''提取policy'''
#此部分代码与上面extract policy相同

'提取policy'

In [27]:
def policy_iter(env):
    num_iter = 1000
    #初始化policy
    policy = np.zeros(env.observation_space.n)
    
    for i in range(num_iter):
        #计算该policy下的value function
        value_function = compute_value_fn(policy)
        #根据得到的value function提取policy
        new_policy = extract_policy(value_function)
        #判断是否为optimal policy
        if (np.all(policy == new_policy)):
            break
        
        policy = new_policy
    
    return policy

In [28]:
optimal_policy = policy_iter(env)
print(optimal_policy)

[0. 3. 3. 3. 0. 0. 0. 0. 3. 1. 0. 0. 0. 2. 1. 0.]


#### 在计算状态值函数的时候，我们是采用迭代计算的方法逐步近似到真实值

#### 通过上述算法可知，我们要想计算V和Q function 就必须知道状态转移矩阵，也就是说环境的动态变化必须是已知的，所以DP算法也是model-based算法